In [ ]:
%%sh
pip install -qU pip sagemaker

In [ ]:
import json
import pprint

import boto3
from IPython.display import Image, Markdown, display

import sagemaker
from sagemaker.huggingface import (HuggingFaceModel,
                                   get_huggingface_llm_image_uri)

In [ ]:
from sagemaker_streaming import print_event_stream

In [ ]:
role = sagemaker.get_execution_role()

# Model Deployment

In [ ]:
model_id = "arcee-ai/arcee-lite"

instance_type = "ml.c6i.4xlarge"

hub = {"HF_MODEL_ID": model_id, "SM_NUM_CPUS": "16", "MESSAGES_API_ENABLED": "true"}

In [ ]:
huggingface_model = HuggingFaceModel(
    image_uri=get_huggingface_llm_image_uri("huggingface"), env=hub, role=role
)

predictor = huggingface_model.deploy(
    initial_instance_count=1,
    instance_type=instance_type,
    container_startup_health_check_timeout=300,
)

# Model Inference

In [ ]:
body = {
    "model": "tgi",
    "messages": [
        {
            "role": "system",
            "content": "As a friendly technical assistant engineer, answer the question in detail.",
        },
        {"role": "user", "content": "Why are transformers better models than LSTM?"},
    ],
    "max_tokens": 1024,
}

In [ ]:
%%time
# Default settings
# Only added to jump back and forth between batch and streaming
from sagemaker.base_deserializers import JSONDeserializer

predictor.deserializer = JSONDeserializer()

response = predictor.predict(body)

In [ ]:
pprint.pprint(response)

In [ ]:
display(Markdown(response["choices"][0]["message"]["content"]))

# Model Inference with Streaming Response

In [ ]:
body = {
    "model": "tgi",
    "messages": [
        {
            "role": "system",
            "content": "As a friendly technical assistant engineer, answer the question in detail.",
        },
        {"role": "user", "content": "Why are transformers better models than LSTM?"},
    ],
    "stream": True,
    "max_tokens": 1024,
}

In [ ]:
smr = boto3.client("sagemaker-runtime")

response = smr.invoke_endpoint_with_response_stream(
    EndpointName=predictor.endpoint_name,
    Body=json.dumps(body),
    ContentType="application/json",
)
print_event_stream(response["Body"])

# Model Inference with Streaming Response and additional context

In [ ]:
Image("chatgpt.png")

In [ ]:
body["messages"] = [
    {
        "role": "system",
        "content": "As a friendly technical assistant engineer, answer the question in detail.",
    },
    {"role": "user", "content": "Is cybertron the ancestor of deep learning?"},
]

response = smr.invoke_endpoint_with_response_stream(
    EndpointName=predictor.endpoint_name,
    Body=json.dumps(body),
    ContentType="application/json",
)
print_event_stream(response["Body"])

In [ ]:
with open("machine-learning-wikipedia.txt") as file:
    context = file.read()

In [ ]:
body["messages"] = [
    {
        "role": "system",
        "content": "As a friendly technical assistant engineer, use the provided context to answer the question in detail.",
    },
    {
        "role": "user",
        "content": f"Is cybertron the ancestor of deep learning? Context: {context}",
    },
]

response = smr.invoke_endpoint_with_response_stream(
    EndpointName=predictor.endpoint_name,
    Body=json.dumps(body),
    ContentType="application/json",
)
print_event_stream(response["Body"])

In [ ]:
predictor.delete_model()
predictor.delete_endpoint()